In [ ]:
# given a name to the data to be read in
datafile = "your_data_here"

import pandas as pd
import numpy as np

In [ ]:
# reading a csv file
data = pd.read_csv(table)
data


In [ ]:
#Checking out the data looks like
data.info()

In [ ]:
# Using our Unique Key column as an index
data = pd.read_csv(table, index_col = 'Unique Key')

In [ ]:
#showing the first ten rows
data.iloc[1:10]

In [ ]:
# checking the values in a single column
data['Incident Zip'].unique()

In [ ]:
# A function to drop rows with NaN and mixed values using exceotion handler
def fix_zip(input_zip):
    try:
        input_zip = int(float(input_zip))
    except:
        try:
            input_zip = int(input_zip.split('-')[0])
        except:
            return np.NaN
    if input_zip < 10000 or input_zip > 19999:
        return np.NaN
    return str(input_zip)

# testing the code above
fix_zip('11211.00')

In [ ]:
#Applying the function to other elements
data['Incident Zip'] = data['Incident Zip'].apply(fix_zip)
data['Incident Zip'].unique()

In [ ]:
# Finally droppng rows with zip == NaN
data = data[data['Incident Zip'].notnull()]
data

In [ ]:
# Checking the data again
data.info()

In [ ]:
# Looking at the Unspecified column under Borough giving us info of the Agency and Incident Zip
data[data['Borough']=='Unspecified'][['Agency','Incident Zip']]

In [ ]:
# checking what wrong with the NYPD complaint if it has significant effect
nypd_complaints_total = data[data['Agency'] == 'NYPD']['Borough'].count()
nypd_unspecified = data[(data['Borough']=='Unspecified') & (data['Agency']=="NYPD")]['Borough'].count()
percentage = nypd_unspecified/nypd_complaints_total*100
print("%1.2f"%percentage)
print(nypd_complaints_total)
print(nypd_unspecified)

In [ ]:
# Checking the rest of the Borough that are Unspecified column
data = data[data['Borough'] != 'Unspecified']
data.info()

In [ ]:
# Now writing all above in a function

def read_311_data(datafile):
    import pandas as pd
    import numpy as np
    #Add the fix_zip function
    def fix_zip(input_zip):
        try:
            input_zip = int(float(input_zip))
        except:
            try:
                input_zip = int(input_zip.split('-')[0])
            except:
                return np.NaN
        if input_zip < 10000 or input_zip > 19999:
            return np.NaN
        return str(input_zip)
    
    #Read the file
    df = pd.read_csv(datafile,index_col='Unique Key')
    
    #fix the zip
    df['Incident Zip'] = df['Incident Zip'].apply(fix_zip)
    
    #drop all rows that have any nans in them (note the easier syntax!)
    
    df = df.dropna(how='any')
    
    #get rid of unspecified boroughs
    df = df[df['Borough'] != 'Unspecified']
    
    #Convert times to datetime and create a processing time column
    
    import datetime
    df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['Closed Date'] = df['Closed Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['processing_time'] =  df['Closed Date'] - df['Created Date']
    
    #Finally, get rid of negative processing times and return the final data frame
    
    df = df[df['processing_time']>=datetime.timedelta(0,0,0)]
    
    return df

df = read_311_data('nyc_311_data_subset.csv')
df.info()

In [ ]:
# looking at the summary
data['processing_time'].describe()